1. Move your annotations to data/custom/labels/. 
2. The dataloader expects that the annotation file corresponding to the image data/custom/images/train.jpg has the path data/custom/labels/train.txt. 
2.1. Each row in the annotation file should define one bounding box, using the syntax label_idx x_center y_center width height. 
The coordinates should be scaled [0, 1], 
and the label_idx should be zero-indexed and correspond to the row number of the class name in data/custom/classes.names.

## Load Annotation

In [11]:
def xyxy2xyhw(bbox, width, height):
    # 归一化
    flag = True
    bbox[0] = bbox[0]/width
    bbox[1] = bbox[1]/height
    bbox[2] = bbox[2]/width
    
    bbox[3] = bbox[3]/height
    
    if(bbox[2]>1 or bbox[3]>1):
        flag = False
    
    ##
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x_center = bbox[0] + w / 2
    y_center = bbox[1] + h / 2
    return [x_center, y_center, w, h], flag

source_root = '/store/dataset/rubbish/VOC2007/'
des_root = '/store/dataset/rubbish_yolo/labels'
import os
imagesetPath = os.path.join(source_root, 'ImageSets', 'Main', 'trainval.txt')

import numpy as np
imagenames = np.loadtxt(imagesetPath, np.str)

from xml.etree import ElementTree as ET
AnnotationRoot = os.path.join(source_root, 'Annotations')

classnames = list(np.loadtxt("classes.names", np.str))
print(classnames)

for imagename in imagenames:
    desPath = os.path.join(des_root, imagename+'.txt')
    AnnotationPath = os.path.join(AnnotationRoot, imagename+'.xml')
    tree = ET.parse(AnnotationPath)
    height = float(tree.findall("./size/height")[0].text)
    width = float(tree.findall('./size/width')[0].text)
    #print(height, width)
    
    with open(desPath, 'w') as f:
        for obj in tree.findall("object"):
            cls = obj.find("name").text
            cls_ind = classnames.index(cls)
            #print("cls_name:%s, cls_ind:%d"%(cls, cls_ind))
            bbox = obj.find("bndbox")
            bbox = [float(bbox.find(x).text) for x in ["xmin", "ymin","xmax","ymax"]]
            ##xyxy=>xyhw
            bbox, flag = xyxy2xyhw(bbox,width, height)
            if flag==False:
                print(desPath)
            saved = np.concatenate((np.array(cls_ind)[np.newaxis], np.array(bbox)), 0)
            #print(saved)
            saved = saved[:,np.newaxis].transpose()
            np.savetxt(f, saved, fmt="%.1f %.6f %.6f %.6f %.6f")


['一次性快餐盒', '书籍纸张', '充电宝', '剩饭剩菜', '包', '垃圾桶', '塑料器皿', '塑料玩具', '塑料衣架', '大骨头', '干电池', '快递纸袋', '插头电线', '旧衣服', '易拉罐', '枕头', '果皮果肉', '毛绒玩具', '污损塑料', '污损用纸', '洗护用品', '烟蒂', '牙签', '玻璃器皿', '砧板', '筷子', '纸盒纸箱', '花盆', '茶叶渣', '菜帮菜叶', '蛋壳', '调料瓶', '软膏', '过期药物', '酒瓶', '金属厨具', '金属器皿', '金属食品罐', '锅', '陶瓷器皿', '鞋', '食用油桶', '饮料瓶', '鱼骨']


In [7]:
import torch

a = torch.cuda.BoolTensor(1,2,3).fill_(0)
a

tensor([[[False, False, False],
         [False, False, False]]], device='cuda:0')

In [38]:
def load_classes(path):
    """
    Loads class labels at 'path'
    """
    fp = open(path, "r")
    names = fp.read().strip().split("\n")
    return names

path = "/home/lrh/program/git/object_detection/code/PyTorch-YOLOv3/data/coco.names"

path = "/store/dataset/rubbish_yolo/classes.names"
classes = load_classes(path)
print(len(classes))
print(classes)

44
['一次性快餐盒', '书籍纸张', '充电宝', '剩饭剩菜', '包', '垃圾桶', '塑料器皿', '塑料玩具', '塑料衣架', '大骨头', '干电池', '快递纸袋', '插头电线', '旧衣服', '易拉罐', '枕头', '果皮果肉', '毛绒玩具', '污损塑料', '污损用纸', '洗护用品', '烟蒂', '牙签', '玻璃器皿', '砧板', '筷子', '纸盒纸箱', '花盆', '茶叶渣', '菜帮菜叶', '蛋壳', '调料瓶', '软膏', '过期药物', '酒瓶', '金属厨具', '金属器皿', '金属食品罐', '锅', '陶瓷器皿', '鞋', '食用油桶', '饮料瓶', '鱼骨']


## train val split

In [ ]:
source_root = '/store/dataset/rubbish/VOC2007/'
des_root = '/store/dataset/rubbish_yolo/'
import os
trainimagesetPath = os.path.join(source_root, 'ImageSets', 'Main', 'train.txt')
testimagesetPath = os.path.join(source_root, 'ImageSets', 'Main', 'train.txt')

trainfilepaths = [os.path.join(des_root, 'images', filename+'.jpg') for filename in list(np.loadtxt(trainimagesetPath, np.str))]

In [ ]:
traintxt = os.path.join(des_root, 'train.txt')
print(traintxt)
print(len(trainfilepaths))
with open(traintxt, 'w') as f:
    np.savetxt(f,np.array(trainfilepaths),fmt='%s')

In [ ]:
import numpy as np
source_root = '/store/dataset/rubbish/VOC2007/'
des_root = '/store/dataset/rubbish_yolo/'
import os

valimagesetPath = os.path.join(source_root, 'ImageSets', 'Main', 'val.txt')
valfilepaths = [os.path.join(des_root, 'images', filename+'.jpg') for filename in list(np.loadtxt(valimagesetPath, np.str))]

valtxt = os.path.join(des_root, 'valid.txt')
print(valtxt)
print(len(valfilepaths))
with open(valtxt, 'w') as f:
    np.savetxt(f,np.array(valfilepaths),fmt='%s')

## Load data from coco for augmentation

#### 1. 生成 trainval.txt

In [66]:
import numpy as np
trainvaltxt = '/store/dataset/rubbish/coco2014/ImageSets/Main/trainval.txt'

imagenames = [imagename.split('.')[0] for imagename in os.listdir('/store/dataset/rubbish/coco2014/JPEGImages')]

with open(trainvaltxt, 'w') as f:
    np.savetxt(f, np.array(imagenames), fmt='%s')

#### 2. 转化成 训练的 格式

In [73]:
def xyxy2xyhw(bbox, width, height):
    # 归一化
    flag = True
    bbox[0] = bbox[0]/width
    bbox[1] = bbox[1]/height
    bbox[2] = bbox[2]/width
    
    bbox[3] = bbox[3]/height
    
    if(bbox[2]>1 or bbox[3]>1):
        flag = False
    
    ##
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x_center = bbox[0] + w / 2
    y_center = bbox[1] + h / 2
    return [x_center, y_center, w, h], flag

source_root = '/store/dataset/rubbish/coco2014/'
des_root = '/store/dataset/rubbish_yolo/coco/labels'
import os
imagesetPath = os.path.join(source_root, 'ImageSets', 'Main', 'trainval.txt')

##复制images文件夹 和 trainval.txt
import shutil
shutil.copytree(os.path.join(source_root, 'JPEGImages'), os.path.join(des_root, '../images'))
shutil.copy(imagesetPath, os.path.join(des_root, '../trainval.txt'))

import numpy as np
imagenames = np.loadtxt(imagesetPath, np.str)

from xml.etree import ElementTree as ET
AnnotationRoot = os.path.join(source_root, 'Annotations')

classnames = list(np.loadtxt("/store/dataset/rubbish/train_classes.txt", np.str))
print(classnames)

for imagename in imagenames:
    desPath = os.path.join(des_root, imagename+'.txt')
    AnnotationPath = os.path.join(AnnotationRoot, imagename+'.xml')
    tree = ET.parse(AnnotationPath)
    height = float(tree.findall("./size/height")[0].text)
    width = float(tree.findall('./size/width')[0].text)
    #print(height, width)
    
    with open(desPath, 'w') as f:
        for obj in tree.findall("object"):
            cls = obj.find("name").text
            if cls == '书籍':
                cls = '书籍纸张'
            cls_ind = classnames.index(cls)
            #print("cls_name:%s, cls_ind:%d"%(cls, cls_ind))
            bbox = obj.find("bndbox")
            bbox = [float(bbox.find(x).text) for x in ["xmin", "ymin","xmax","ymax"]]
            ##xyxy=>xyhw
            bbox, flag = xyxy2xyhw(bbox,width, height)
            if flag==False:
                print(desPath)
            saved = np.concatenate((np.array(cls_ind)[np.newaxis], np.array(bbox)), 0)
            #print(saved)
            saved = saved[:,np.newaxis].transpose()
            np.savetxt(f, saved, fmt="%.1f %.6f %.6f %.6f %.6f")

['一次性快餐盒', '书籍纸张', '充电宝', '剩饭剩菜', '包', '垃圾桶', '塑料器皿', '塑料玩具', '塑料衣架', '大骨头', '干电池', '快递纸袋', '插头电线', '旧衣服', '易拉罐', '枕头', '果皮果肉', '毛绒玩具', '污损塑料', '污损用纸', '洗护用品', '烟蒂', '牙签', '玻璃器皿', '砧板', '筷子', '纸盒纸箱', '花盆', '茶叶渣', '菜帮菜叶', '蛋壳', '调料瓶', '软膏', '过期药物', '酒瓶', '金属厨具', '金属器皿', '金属食品罐', '锅', '陶瓷器皿', '鞋', '食用油桶', '饮料瓶', '鱼骨']


## 将数据合并生成三个新的训练数据集 trainval.txt, train+coco1000.txt, trainval+coco1000.txt

### 1. trainval.txt 合并 train.txt 和 val.txt

In [21]:
import os
txtroot = '/store/dataset/rubbish_yolo'
def combine2txt(filename1, filename2):
    traintxt = os.path.join(txtroot, filen)
    validtxt = os.path.join(txtroot, 'valid.txt')

    with open(traintxt, 'r') as f:
        trainpaths = np.loadtxt(f, np.str)

    with open(validtxt, 'r') as f:
        validpaths = np.loadtxt(f, np.str)

    trainvalpaths = []

    trainvalpaths.extend(trainpaths)
    trainvalpaths.extend(validpaths)

    print(len(trainvalpaths))

    with open(os.path.join(txtroot, 'trainval.txt'), 'w') as f:
        np.savetxt(f, np.array(trainvalpaths), fmt='%s')
combine2txt('train.txt', 'valid.txt', )

14964


### 2. 生成coco1000.txt

In [72]:
import os
txtroot = '/store/dataset/rubbish_yolo'
traintxt = os.path.join(txtroot, 'train.txt')
cocotxt = os.path.join(txtroot, 'coco/trainval.txt')

with open(cocotxt, 'r') as f:
    imagepaths = [os.path.join(os.path.dirname(cocotxt), 'images', imagename+'.jpg') 
                  for imagename in np.loadtxt(f, np.str)]

labelpaths = [imagepath.replace('images', 'labels').replace('.jpg', '.txt') for imagepath in imagepaths]

#print(labelpaths)
num_c = dict()
num_c['1'] = 0
num_c['35'] = 0
num_c['27'] = 0


cocoimagepaths = []

thres = 1000

for ind, labelpath in enumerate(labelpaths):
    label = np.loadtxt(labelpath, np.str).astype(np.float)
    if label.ndim<2:
        label = label[np.newaxis, :]
    cls = np.unique(label[:, 0])
    for c in cls:
        c = str(int(c))
        if num_c[c]>=thres:
            ## 如果其他类没有到达边界，则也保存该图像
            continue;
            #break;
        else:
            ## 将 该图像 包含到 文件中
            num_c[c] += 1
            cocoimagepaths.append(imagepaths[ind])
print(num_c)
            
with open(os.path.join(txtroot, 'coco1000.txt'), 'w') as f:
    np.savetxt(f, np.array(cocoimagepaths), fmt='%s')

{'1': 1000, '35': 1000, '27': 1000}
